# Feature Visualizations

In [1]:
!pip install tensorflow==2.0.0

     |████████████████████████████████| 86.3MB 1.4MB/s 
     |████████████████████████████████| 450kB 35.1MB/s 
     |████████████████████████████████| 3.8MB 25.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=3fa1587ebda51b24cbc1b82acb5de94bcd5df7ed91560361c10b8827243e1cdb
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: gast 0.3.2
    Uninstalling gast-0.3.2:
      Successfully uninstalled gast-0.3.2
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


## Required Packages

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adadelta, Adam
from tensorflow.keras.layers import Lambda, Input
from tensorflow.keras import regularizers
from tensorflow.keras import Model
import tensorflow as tf

from progressbar import ProgressBar, Bar, Percentage
from skimage.restoration import denoise_bilateral
from skimage.transform import resize
from skimage.filters import gaussian
from imageio import imwrite, imread
import numpy as np
import sys
import os

## Init

In [0]:
def make_directories():
    if not os.path.exists('docs'):
        os.mkdir('docs')

    if not os.path.exists('bin'):
        os.mkdir('bin')

    if not os.path.exists(os.path.join('bin', 'shapes')):
        os.mkdir(os.path.join('bin', 'shapes'))

make_directories()

## Utilities

In [0]:
# referenced from https://github.com/tensorflow/lucid/blob/master/lucid/optvis/overrides/redirected_relu_grad.py

@tf.RegisterGradient('RedirectedRelu')
def redirected_relu_grad(op, grad):
    x = op.inputs[0]
    relu_grad = tf.where(x < 0., tf.zeros_like(grad), grad)
    neg_pushing_lower = tf.logical_and(x < 0., grad > 0.)
    redirected_grad = tf.where(neg_pushing_lower, tf.zeros_like(grad), grad)
    assert_op = tf.Assert(tf.greater(tf.rank(relu_grad), 1), [tf.rank(relu_grad)])
    with tf.control_dependencies([assert_op]):
        batch = tf.shape(relu_grad)[0]
        reshaped_relu_grad = tf.reshape(relu_grad, [batch, -1])
        relu_grad_mag = tf.norm(reshaped_relu_grad, axis=1)
    result_grad = tf.where(relu_grad_mag > 0., relu_grad, redirected_grad)
    return result_grad

@tf.RegisterGradient('RedirectedRelu6')
def redirected_relu6_grad(op, grad):
    x = op.inputs[0]
    relu6_cond = tf.logical_or(x < 0., x > 6.)
    relu_grad = tf.where(relu6_cond, tf.zeros_like(grad), grad)
    neg_pushing_lower = tf.logical_and(x < 0., grad > 0.)
    pos_pushing_higher = tf.logical_and(x > 6., grad < 0.)
    dir_filter = tf.logical_or(neg_pushing_lower, pos_pushing_higher)
    redirected_grad = tf.where(dir_filter, tf.zeros_like(grad), grad)
    assert_op = tf.Assert(tf.greater(tf.rank(relu_grad), 1), [tf.rank(relu_grad)])
    with tf.control_dependencies([assert_op]):
        batch = tf.shape(relu_grad)[0]
        reshaped_relu_grad = tf.reshape(relu_grad, [batch, -1])
        relu_grad_mag = tf.norm(reshaped_relu_grad, axis=1)
    result_grad = tf.where(relu_grad_mag > 0., relu_grad, redirected_grad)
    return result_grad

In [0]:
def get_model(model_name):
    model = None
    if model_name == 'mobile-net-v1':
        from tensorflow.keras.applications.mobilenet import MobileNet
        model = MobileNet(weights='imagenet', include_top=False,
                          input_shape=(None, None, 3))
    elif model_name == 'mobile-net-v2':
        from keras.applications.mobilenet_v2 import MobileNetV2
        model = MobileNetV2(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'vgg-16':
        from keras.applications.vgg16 import VGG16
        model = VGG16(weights='imagenet', include_top=False,
                      input_shape=(None, None, 3))
    elif model_name == 'vgg-19':
        from keras.applications.vgg19 import VGG19
        model = VGG19(weights='imagenet', include_top=False,
                      input_shape=(None, None, 3))
    elif model_name == 'densenet-121':
        from keras.applications.densenet import DenseNet121
        model = DenseNet121(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'densenet-169':
        from keras.applications.densenet import DenseNet169
        model = DenseNet169(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'densenet-201':
        from keras.applications.densenet import DenseNet201
        model = DenseNet201(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'resnet-50':
        from keras.applications.resnet50 import ResNet50
        model = ResNet50(weights='imagenet', include_top=False,
                         input_shape=(None, None, 3))
    elif model_name == 'xception':
        from keras.applications.xception import Xception
        model = Xception(weights='imagenet', include_top=False,
                         input_shape=(None, None, 3))
    elif model_name == 'inception-v3':
        from keras.applications.inception_v3 import InceptionV3
        model = InceptionV3(weights='imagenet', include_top=False,
                            input_shape=(None, None, 3))
    elif model_name == 'inceptionresnet-v2':
        from keras.applications.inception_resnet_v2 import InceptionResNetV2
        model = InceptionResNetV2(weights='imagenet', include_top=False,
                                  input_shape=(None, None, 3))
    return model